In [1]:




# #Defining our hnswlib index
# index_path = "./hnswlib.index"
# #We use Inner Product (dot-product) as Index. We will normalize our vectors to unit length, then is Inner Product equal to cosine similarity
# index = hnswlib.Index(space = 'cosine', dim = embedding_size)

# if os.path.exists(index_path):
#     print("Loading index...")
#     index.load_index(index_path)
# else:
#     ### Create the HNSWLIB index
#     print("Start creating HNSWLIB index")
#     index.init_index(max_elements = len(corpus_embeddings), ef_construction = 400, M = 64)

#     # Then we train the index to find a suitable clustering
#     index.add_items(corpus_embeddings, list(range(len(corpus_embeddings))))

#     print("Saving index to:", index_path)
#     index.save_index(index_path)

# # Controlling the recall by setting ef:
# index.set_ef(50)  # ef should always be > top_k_hits

# ######### Search in the index ###########

# print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

# while True:
#     inp_question = input("Please enter a question: ")

#     start_time = time.time()
#     question_embedding = model.encode(inp_question)

#     #We use hnswlib knn_query method to find the top_k_hits
#     corpus_ids, distances = index.knn_query(question_embedding, k=top_k_hits)

#     # We extract corpus ids and scores for the first query
#     hits = [{'corpus_id': id, 'score': 1-score} for id, score in zip(corpus_ids[0], distances[0])]
#     hits = sorted(hits, key=lambda x: x['score'], reverse=True)
#     end_time = time.time()

#     print("Input question:", inp_question)
#     print("Results (after {:.3f} seconds):".format(end_time-start_time))
#     for hit in hits[0:top_k_hits]:
#         print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))

#     # Approximate Nearest Neighbor (ANN) is not exact, it might miss entries with high cosine similarity
#     # Here, we compute the recall of ANN compared to the exact results
#     correct_hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k_hits)[0]
#     correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])

#     ann_corpus_ids = set([hit['corpus_id'] for hit in hits])
#     if len(ann_corpus_ids) != len(correct_hits_ids):
#         print("Approximate Nearest Neighbor returned a different number of results than expected")

#     recall = len(ann_corpus_ids.intersection(correct_hits_ids)) / len(correct_hits_ids)
#     print("\nApproximate Nearest Neighbor Recall@{}: {:.2f}".format(top_k_hits, recall * 100))

#     if recall < 1:
#         print("Missing results:")
#         for hit in correct_hits[0:top_k_hits]:
#             if hit['corpus_id'] not in ann_corpus_ids:
#                 print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))
#     print("\n\n========\n")


In [2]:
"""
This example uses Approximate Nearest Neighbor Search (ANN) with Hnswlib  (https://github.com/nmslib/hnswlib/).
Install it with `pip install hnswlib`

For an embeddings model, we use the SBERT model 'msmarco-distilbert-base-v4'
For a dataset we use our own; gives us a way to compare with naive search
"""

"\nThis example uses Approximate Nearest Neighbor Search (ANN) with Hnswlib  (https://github.com/nmslib/hnswlib/).\nInstall it with `pip install hnswlib`\n\nFor an embeddings model, we use the SBERT model 'msmarco-distilbert-base-v4'\nFor a dataset we use our own; gives us a way to compare with naive search\n"

In [3]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import pickle
import time
import hnswlib
import pandas as pd
import os

/home/auro/anaconda3/envs/openai/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = 'msmarco-distilbert-base-v4'
embeddings_model = SentenceTransformer(model_name)
embedding_size = 768    #Size of embeddings
top_k_hits = 5         #Output k hits


In [9]:
embeddings_cache_path = f"dishwasher-repair-manual-embeddings-{model_name.replace('/', '_')}.pkl"
print(embeddings_cache_path)

#Check if embeddings cache path exists
if not os.path.exists(embeddings_cache_path):
    # Check if the dataset exists. If not, read them
    df = pd.read_csv('../dish-washer-data.csv')
    df["tokens"] = pd.to_numeric(df["tokens"])  # convert column "tokens" of a DataFrame
    df = df.set_index(["title", "heading"])
    print(f"{len(df)} rows in the data.")
    df.sample(10)
    # TODO get some stats on max/min content length
    corpus_size = df.shape[0]

    print("Encode the corpus. This might take a while...")
    # This could take a bit of time
    # document_embeddings = compute_doc_embeddings(df, embeddings_model)
    corpus_embeddings = embeddings_model.encode(list(df), show_progress_bar=True, convert_to_numpy=True)

    print("Store file on disc")
    with open(embeddings_cache_path, "wb") as fOut:
        pickle.dump({'sentences': list(df), 'embeddings': corpus_embeddings}, fOut)
else:
    print("Loading pre-computed embeddings from disc")
    with open(embeddings_cache_path, "rb") as fIn:
        cache_data = pickle.load(fIn)
        corpus_sentences = cache_data['sentences']
        corpus_embeddings = cache_data['embeddings']
        print("Loafing done")

dishwasher-repair-manual-embeddings-msmarco-distilbert-base-v4.pkl
Loading pre-computed embeddings from disc
Loafing done
